In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
import json
import time
import requests

url = 'https://stat.molit.go.kr/portal/cate/statView.do?hRsId=58&hFormId=5498&hSelectId=5498&hPoint=00&hAppr=1&hDivEng=&oFileName=&rFileName=&midpath=&sFormId=5498&sStart=201101&sEnd=202502&sStyleNum=2&settingRadio=xlsx'
driver = webdriver.Chrome()
#chrome 브라우저열기
driver.get(url)
time.sleep(1)




for i in range(2011, 2026):
    for j in range(1, 13):

        if (i==2018 and j>9) or i>2018 :     #201810부터는 총계가 맨 아래로 감.
            # 기간선택
            from_year_month = Select(driver.find_element(By.XPATH, '//*[@id="sStart"]'))
            from_year_month.select_by_value(str(i)+str(j).zfill(2))
            time.sleep(2)
            
            to_year_month = Select(driver.find_element(By.XPATH, '//*[@id="sEnd"]'))
            to_year_month.select_by_value(str(i)+str(j).zfill(2))
            time.sleep(2)

            # 조회버튼
            # 조회버튼 클릭
            button = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div[3]/div/div[1]/div/button')
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
            time.sleep(2)
            button.click()
            time.sleep(5)
            
            # 데이터 파싱
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            # 컬럼명 가져오기
            th_list = soup.select('#sheet01-table > tbody > tr:nth-child(1) > td:nth-child(2) > div > table > tbody > tr:nth-child(1) > th')
            span_list = [th.get_text(strip=True) for th in th_list]
            span_list[0:3] = ["월(Monthly)", "시도명", "시군구"]  # 첫 3개 컬럼명 변경
            
            # 데이터 가져오기
            tbody = soup.select('#sheet01-table > tbody > tr:nth-child(2) > td:nth-child(1) > div > div.GMPageOne > table > tbody')
            tr_rows = tbody[0].find_all('tr', onmousemove=True)
            
            # 데이터 리스트
            tr_texts = [tr.get_text(strip=True) for tr in tr_rows]
            #print(tr_texts)
            
            
            
            # 모든 숫자 데이터를 저장할 리스트
            
            values_tbody = soup.select('#sheet01-table > tbody > tr:nth-child(2) > td:nth-child(2) > div > div.GMPageOne > table > tbody')
            values_tr_rows = values_tbody[0].find_all('tr', onmousemove=True)
            
            # 모든 숫자 데이터를 저장할 리스트
            number_lists = []
            for values_tr in values_tr_rows:
                number_list = []
                for td in values_tr.find_all('td'):
                    num_text = td.get_text(strip=True).replace(',', '')  # 공백 제거 및 ',' 제거
                    if num_text.isdigit():  # 숫자인 경우만 추가
                        number_list.append(int(num_text))
                number_lists.append(number_list[:-4])
            
            # 시군구별 데이터
            values_data = json.dumps(number_lists, indent=4, ensure_ascii=False)
            #print(number_lists)
            
            
            #  최종 저장할 딕셔너리
            # 최종 저장할 딕셔너리
            final_result_dict = {}
            
            # 첫 번째 요소 처리
            year_month = tr_texts[0][:7]  # "2018-10"
            city_name = tr_texts[0][7:9]  # "서울"
            first_district = tr_texts[0][9:]  # "강남구"
            
            #  시도(city) 초기화
            final_result_dict[year_month] = {city_name: {}}
            
            #  첫 번째 시군구에도 데이터 넣기 (밀리는 문제 해결)
            district_index = 0
            if district_index < len(number_lists):
                final_result_dict[year_month][city_name][first_district] = number_lists[district_index]
                district_index += 1  # 첫 번째 값 처리 완료
            
            #  나머지 시군구 데이터 추가
            i = 1
            while i < len(tr_texts):
                text = tr_texts[i]
            
                if text == "계":  
                    #  "계" 추가 (데이터 포함!)
                    if district_index < len(number_lists):
                        final_result_dict[year_month][city_name]["계"] = number_lists[district_index]
                        district_index += 1  # "계"도 데이터가 있을 경우 넘어감
            
                    # "계" 다음에 새로운 시도가 있는지 확인
                    if i + 1 < len(tr_texts):  
                        next_text = tr_texts[i + 1]
                        new_city_name = next_text[:2]  
                        new_district_name = next_text[2:]  
            
                        if new_city_name not in final_result_dict[year_month]:  #  새로운 시도 초기화
                            final_result_dict[year_month][new_city_name] = {}
            
                        if district_index < len(number_lists):
                            final_result_dict[year_month][new_city_name][new_district_name] = number_lists[district_index]
                            district_index += 1  # 다음 데이터로 이동
            
                        city_name = new_city_name  #  새 시도로 변경
                        i += 1  # 추가된 데이터 건너뛰기
            
                else:
                    #  기존 시도(city)가 없으면 초기화
                    if city_name not in final_result_dict[year_month]:
                        final_result_dict[year_month][city_name] = {}
            
                    #  기존 시군구(district)가 없으면 초기화
                    if text not in final_result_dict[year_month][city_name]:
                        final_result_dict[year_month][city_name][text] = []
            
                    #  값이 존재하면 추가
                    if district_index < len(number_lists):
                        final_result_dict[year_month][city_name][text] = number_lists[district_index]
                        district_index += 1  # number_lists의 인덱스 증가
            
                i += 1
            
            #print(json.dumps(final_result_dict, ensure_ascii=False, indent=4))
            print(final_result_dict)

            
        else :
            
            # 기간선택
            from_year_month = Select(driver.find_element(By.XPATH, '//*[@id="sStart"]'))
            from_year_month.select_by_value(str(i)+str(j).zfill(2))
            time.sleep(2)
            
            to_year_month = Select(driver.find_element(By.XPATH, '//*[@id="sEnd"]'))
            to_year_month.select_by_value(str(i)+str(j).zfill(2))
            time.sleep(2)
    
    
            # 조회버튼
            button = driver.find_element(By.XPATH,'//*[@id="main"]/div/div[2]/div[2]/div[3]/div/div[1]/div/button')
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
            time.sleep(2)
            button.click()
            time.sleep(5)
    
    
            # 데이터 파싱
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
    
            # 날짜, 시도명, total, 시군구 컬럼's 리스트 만들기
            th_list = soup.select('#sheet01-table > tbody > tr:nth-child(1) > td:nth-child(2) > div > table > tbody > tr:nth-child(1) > th')
            span_list = []
            for th in th_list :
                spans = th.find_all("span")
                for span in spans:
                    # 텍스트 출력
                    span_list.append(span.text.strip())       
            del span_list[-1]  # 불필요한 데이터 삭제
            span_list[0:3] = ["월(Monthly)", "시도명", "시군구"]  # [0:3] 컬럼명 변경
            # print(span_list) # 확인용
    
    
            tbody = soup.select('#sheet01-table > tbody > tr:nth-child(2) > td:nth-child(1) > div > div.GMPageOne > table > tbody')
            tr_rows = tbody[0].find_all('tr', onmousemove=True)
            
            
            # 날짜, 시도명, total, 시군구 컬럼's 리스트 완성
            tr_texts = [tr.get_text(strip=True) for tr in tr_rows]
            #print(tr_texts) # 확인용
            tr_texts.insert(1, tr_texts[0][7:])   # 시도명 ,  ex)서울
            tr_texts[0] = tr_texts[0][:7]           # 날짜 date, ex)2011-01
            #print(tr_texts) # 확인용
    
    
            # values_data 구현
    
            # 모든 숫자 데이터를 저장할 리스트
            
            values_tbody = soup.select('#sheet01-table > tbody > tr:nth-child(2) > td:nth-child(2) > div > div.GMPageOne > table > tbody')
            values_tr_rows = values_tbody[0].find_all('tr', onmousemove=True)
            
            # 모든 숫자 데이터를 저장할 리스트
            number_lists = []
            for values_tr in values_tr_rows:
                number_list = []
                for td in values_tr.find_all('td'):
                    num_text = td.get_text(strip=True).replace(',', '')  # 공백 제거 및 ',' 제거
                    if num_text.isdigit():  # 숫자인 경우만 추가
                        number_list.append(int(num_text))
                number_lists.append(number_list[:-4])
    
            # 시군구별 데이터
            values_data = json.dumps(number_lists, indent=4, ensure_ascii=False)
    
    
        
            #시도명 : [시군구list] 만들기      
            result_dict = {}       #변환된 데이터를 저장할 딕셔너리
            final_result_dict = {} #날짜별 데이터를 저장할 딕셔너리
            
            year_month = tr_texts[0] # 현재날짜 (예: "2011-01")
            
            exclude_words = ['계양구', '계룡시']
            
            current_key = None  # 현재 키 (예: "서울")
            num_index = 0  # 숫자 리스트의 인덱스
            
            for item in tr_texts[1:]:
                if '계' in item not in exclude_words:  # "계"가 포함된 항목이면 키로 사용
                    current_key = item.replace("계", "").strip()  # "계" 제거 후 키 생성 (예: "서울")
                    result_dict[current_key] = {"total": number_lists[num_index]}  # total 값 설정
                    num_index += 1  # 숫자 리스트 인덱스 증가
                elif current_key and num_index < len(number_lists):  # 지역구 추가
                    result_dict[current_key][item] = number_lists[num_index]  # 지역구에 숫자 매칭
                    num_index += 1  # 숫자 리스트 인덱스 증가
            
            final_result_dict[year_month] = result_dict
            #print(json.dumps(final_result_dict, indent=4, ensure_ascii=False))
            
            print(final_result_dict)
      
driver.quit()

{'2011-01': {'서울': {'강남구': [3969, 2292616, 146971, 2443556, 2493, 157299, 17198, 176990, 3711, 310364, 52648, 366723, 314, 1062, 2528, 3904], '강동구': [163, 192610, 31456, 224229, 86, 8261, 1728, 10075, 161, 17407, 1630, 19198, 21, 94, 134, 249], '강북구': [81, 107074, 6261, 113416, 50, 8159, 703, 8912, 115, 15451, 2096, 17662, 10, 34, 157, 201], '강서구': [103, 55201, 2585, 57889, 47, 5189, 727, 5963, 142, 8639, 1434, 10215, 10, 15, 17, 42], '관악구': [111, 132643, 15523, 148277, 58, 9761, 1254, 11073, 167, 18918, 2672, 21757, 13, 80, 88, 181], '광진구': [146, 94175, 3508, 97829, 90, 7039, 318, 7447, 156, 12204, 1984, 14344, 7, 14, 90, 111], '구로구': [104, 72214, 2723, 75041, 49, 5316, 344, 5709, 153, 10682, 1345, 12180, 7, 20, 42, 69], '금천구': [90, 94744, 17026, 111860, 60, 7085, 1337, 8482, 127, 15877, 2182, 18186, 8, 32, 44, 84], '노원구': [25, 51921, 2160, 54106, 13, 4824, 433, 5270, 59, 11098, 2274, 13431, 1, 30, 39, 70], '도봉구': [90, 124129, 6875, 131094, 60, 8569, 791, 9420, 128, 13402, 1844, 15374